In [1]:
import pickle

with open('preprocessed_data.pkl', 'rb') as f:
    data = pickle.load(f)

for k, v in data.items():
    try:
        print(k, type(v), len(v))
    except:
        print(k, type(v))

train_df <class 'pandas.core.frame.DataFrame'> 342665
val_df <class 'pandas.core.frame.DataFrame'> 47464
test_df <class 'pandas.core.frame.DataFrame'> 123739
train_matrix <class 'scipy.sparse._csr.csr_matrix'>
val_matrix <class 'scipy.sparse._csr.csr_matrix'>
test_matrix <class 'scipy.sparse._csr.csr_matrix'>
user_to_idx <class 'dict'> 98975
idx_to_user <class 'dict'> 98975
business_to_idx <class 'dict'> 28274
idx_to_business <class 'dict'> 28274
business_stats <class 'pandas.core.frame.DataFrame'> 28223
n_users <class 'int'>
n_businesses <class 'int'>


## Baseline 1: Matrix Factorization (MF) Baseline
Matrix Factorization (MF) is a classical baseline for recommendation tasks, especially when explicit ratings are available. Unlike Jaccard (a pure similarity baseline), MF is designed to predict the user’s rating for an item, and can also be used for Top-K ranking by sorting predicted scores.

This MF baseline includes:

1.A complete MF model definition (TensorFlow)

2.Training loop

3.Rating prediction, and transfered to similarity

4.MSE calculation

5.Top-K recommendation

6.Ranking metrics (Recall@K, NDCG@K)

In [31]:
import pandas as pd
df = pd.read_csv("train_data.csv")
print(df.columns)
df.head()

Index(['business_id', 'user_id', 'rating', 'review_text', 'pics', 'user_idx',
       'business_idx'],
      dtype='object')


,business_id,user_id,rating,review_text,pics,user_idx,business_idx
0,6040598a65e4ba0588bb0fca,100000107216850094011,5,The food here is amazing! The curries are rich...,[{'id': 'AF1QipNCVZIuBLkTOzpgwNZz9BFCX0CD3jjNa...,98969,28121
1,604056bbc6fcf1fddba0a0e0,100000107216850094011,3,Great brunch spot with bottomless coffee from ...,[{'id': 'AF1QipPC7hM3F612LEXzVAeIp-UMFtg1fIXWj...,98969,28142
2,6040594f65e4ba0588bb0fa1,100000107216850094011,5,Philadelphia has many excellent restaurants bu...,[{'id': 'AF1QipN0gtINkHKgJaqG-AWsN_0-od2XhMPJu...,98969,28122
3,6042481f2e57ebdea29c95aa,100000149611993816967,5,"Great flavor, fun to share tapas and sushi. Th...",[{'id': 'AF1QipPc19S3uC5eywfMAEs7adk3RD7aaaHLd...,98785,25640
4,6042483cb9a6829e686e8cdd,100000149611993816967,4,Good burger great fries and onion rings,[{'id': 'AF1QipOVdGguJCQddhusocI6-crhk78AeS5ml...,98785,25639


### Part 0 — Data Preparation

In [32]:
import pickle

with open("preprocessed_data.pkl", "rb") as f:
    data = pickle.load(f)

train_df = data["train_df"]
val_df = data["val_df"]
test_df = data["test_df"]

user_to_idx = data["user_to_idx"]
business_to_idx = data["business_to_idx"]

n_users = data["n_users"]
n_businesses = data["n_businesses"]

print(n_users, n_businesses)


98975 28274


### Part 1 — Matrix Factorization Model (TensorFlow, Keras)


In [33]:
import tensorflow as tf

class MF(tf.keras.Model):
    def __init__(self, n_users, n_items, K=32):
        super().__init__()

        self.user_emb = tf.keras.layers.Embedding(n_users, K)
        self.item_emb = tf.keras.layers.Embedding(n_items, K)
        self.user_bias = tf.keras.layers.Embedding(n_users, 1)
        self.item_bias = tf.keras.layers.Embedding(n_items, 1)

        self.global_bias = tf.Variable(0.0)

    def call(self, user_idx, item_idx):
        u = self.user_emb(user_idx)
        i = self.item_emb(item_idx)
        bu = self.user_bias(user_idx)
        bi = self.item_bias(item_idx)

        dot = tf.reduce_sum(u * i, axis=1)
        pred = self.global_bias + tf.squeeze(bu) + tf.squeeze(bi) + dot
        return pred


### Part 2 — Training the MF Model

In [35]:
reg_lambda = 1e-6  # small regularization strength

for epoch in range(epochs):
    idx = np.random.permutation(len(train_df))

    for start in range(0, len(train_df), batch_size):
        end = start + batch_size
        batch = idx[start:end]

        u = tf.gather(user_tensor, batch)
        i = tf.gather(item_tensor, batch)
        r = tf.gather(rating_tensor, batch)

        with tf.GradientTape() as tape:
            pred = model_mf(u, i)

            # MSE loss
            mse_loss = tf.reduce_mean((pred - r) ** 2)

            # L2 Regularization (IMPORTANT)
            reg_loss = (
                reg_lambda * tf.nn.l2_loss(model_mf.user_emb.weights[0]) +
                reg_lambda * tf.nn.l2_loss(model_mf.item_emb.weights[0]) +
                reg_lambda * tf.nn.l2_loss(model_mf.user_bias.weights[0]) +
                reg_lambda * tf.nn.l2_loss(model_mf.item_bias.weights[0])
            )

            # total MF loss
            loss = mse_loss + reg_loss

        grads = tape.gradient(loss, model_mf.trainable_variables)
        optimizer.apply_gradients(zip(grads, model_mf.trainable_variables))

    print(f"Epoch {epoch+1}, loss={loss.numpy():.4f}")



Epoch 1, loss=0.2967
Epoch 2, loss=0.2803
Epoch 3, loss=0.2758
Epoch 4, loss=0.2744
Epoch 5, loss=0.2745


### Part 3: MF is converted into similarity

In [37]:
from sklearn.metrics.pairwise import cosine_similarity

mf_item_emb = model_mf.item_emb.weights[0].numpy()
item_sim_mf = cosine_similarity(mf_item_emb)


In [38]:
def recommend_mf_to_user(user_idx, topk=10):
    item_ids = np.arange(n_businesses)

    # Predict rating for all items for this user
    u = tf.constant([user_idx] * n_businesses, dtype=tf.int32)
    i = tf.constant(item_ids, dtype=tf.int32)
    preds = model_mf(u, i).numpy()

    # Remove seen items
    seen = set(train_df[train_df.user_idx == user_idx].business_idx)
    preds_filtered = [(biz, score) for biz, score in zip(item_ids, preds) if biz not in seen]

    # Sort by predicted score
    preds_sorted = sorted(preds_filtered, key=lambda x: x[1], reverse=True)
    return preds_sorted[:topk]

### Part 4 — Recall@K and NDCG@K (General Ranking Metrics)

In [39]:
import math

def recall_at_k(recommended, ground_truth, K):
    """
    recommended: list of item indices predicted by the model
    ground_truth: list of ground-truth liked items (rating >= 4)
    """
    if len(ground_truth) == 0:
        return None
    hit = len(set(recommended[:K]) & set(ground_truth))
    return hit / len(ground_truth)


def ndcg_at_k(recommended, ground_truth, K):
    """
    Standard NDCG@K for ranking evaluation.
    """
    dcg = 0.0
    for rank, item in enumerate(recommended[:K], start=1):
        if item in ground_truth:
            dcg += 1 / math.log2(rank + 1)

    max_rel = min(K, len(ground_truth))
    idcg = sum(1 / math.log2(rank + 1) for rank in range(1, max_rel + 1))

    return dcg / idcg if idcg > 0 else None


### Part 5 — Global Evaluation for MF

In [40]:
import numpy as np

def evaluate_mf(K=10, sample_users=2000):
    recalls = []
    ndcgs = []

    # Randomly sample users from test set
    users = test_df.user_idx.unique()
    users = np.random.choice(users, size=min(len(users), sample_users), replace=False)

    for u in users:
        # Use MF to recommend top-K items for this user
        recs = [item for item, score in recommend_mf_to_user(u, topk=K)]

        # Ground truth = items rated >= 4 by this user in test set
        truth = list(test_df[(test_df.user_idx == u) & (test_df.rating >= 4)].business_idx)

        # Compute metrics
        r = recall_at_k(recs, truth, K)
        n = ndcg_at_k(recs, truth, K)

        if r is not None:
            recalls.append(r)
        if n is not None:
            ndcgs.append(n)

    # Return average scores
    return np.mean(recalls), np.mean(ndcgs)


In [41]:
recall10, ndcg10 = evaluate_mf(K=10)
print("MF Recall@10 =", recall10)
print("MF NDCG@10 =", ndcg10)

MF Recall@10 = 0.00973370064279155
MF NDCG@10 = 0.006545760831757247


## Baseline 2：Item–Item Jaccard Similarity
This is a fully functional Item–Item Jaccard Similarity baseline implementation, including:

1.Building item–user mappings

2.Computing item–item Jaccard similarity

3.Generating Top-K recommendations for users

4.Computing MSE (not theoretically meaningful, but included if required)

5.Computing Recall@K and NDCG@K (the correct metrics for ranking models)

### Part 0 — Load Data

In [2]:
train_df = data["train_df"]
test_df = data["test_df"]

# Ensure we only keep necessary fields
train_df = train_df[["user_idx", "business_idx", "rating"]]
test_df = test_df[["user_idx", "business_idx", "rating"]]


### Part 1 — Build the Item–User Mapping (Required for Jaccard)

In [3]:
from collections import defaultdict

users_per_item = defaultdict(set)
items_per_user = defaultdict(set)

for _, row in train_df.iterrows():
    u = row.user_idx
    i = row.business_idx
    users_per_item[i].add(u)
    items_per_user[u].add(i)


In [4]:
print("Number of items:", len(users_per_item))
print("Users who visited item 25639:", len(users_per_item[25639]))


Number of items: 28223
Users who visited item 25639: 2


### Part 2 — Jaccard Similarity Function

In [5]:
def jaccard_similarity(i, j):
    users_i = users_per_item[i]
    users_j = users_per_item[j]
    
    if len(users_i) == 0 or len(users_j) == 0:
        return 0.0

    inter = len(users_i & users_j)
    union = len(users_i | users_j)
    return inter / union


### Part 3 — Generate Top-K Recommendations (Item-Based Collaborative Filtering)

In [12]:
def recommend_jaccard_to_user(user_idx, topK=10):
    seen_items = items_per_user[user_idx]
    if len(seen_items) == 0:
        return []

    scores = defaultdict(float)
    candidate_items = set()

    # Step 1: gather candidates = only items visited by users who visited seen items
    for item in seen_items:
        for u in users_per_item[item]:
            for other_item in items_per_user[u]:
                if other_item != item:
                    candidate_items.add(other_item)

    # Step 2: compute similarity only for candidates
    for item in seen_items:
        for other_item in candidate_items:
            if other_item == item:
                continue
            sim = jaccard_similarity(item, other_item)
            scores[other_item] += sim

    # Step 3: remove already-visited items
    for s in seen_items:
        scores.pop(s, None)

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [item for item, score in ranked[:topK]]




In [13]:
recommend_jaccard_to_user(98785, topK=10)


[22409, 4027, 5656, 13959, 14481, 2120, 24244, 25646, 21751, 13593]

### Part 4 — Recall@K and NDCG@K (Correct Metrics for Ranking)

In [14]:
def recall_at_k(recommended, ground_truth, K):
    if len(ground_truth) == 0:
        return None
    recommended_k = recommended[:K]
    hit = len(set(recommended_k) & set(ground_truth))
    return hit / len(ground_truth)

In [15]:
import math

def ndcg_at_k(recommended, ground_truth, K):
    dcg = 0.0
    for rank, item in enumerate(recommended[:K], start=1):
        if item in ground_truth:
            dcg += 1 / math.log2(rank + 1)

    # IDCG: ideal ranking
    max_rel = min(K, len(ground_truth))
    idcg = sum(1 / math.log2(rank + 1) for rank in range(1, max_rel + 1))

    return dcg / idcg if idcg > 0 else None

### Part 5 — Global Evaluation for the Jaccard Baseline

In [16]:
def evaluate_jaccard(K=10, sample_users=2000):
    recalls, ndcgs = [], []

    users = test_df.user_idx.unique()
    users = np.random.choice(users, size=min(len(users), sample_users), replace=False)

    for u in users:
        recs = recommend_jaccard_to_user(u, topK=K)

        # ground truth = items user liked (rating >=4)
        truth = list(test_df[(test_df.user_idx == u) & (test_df.rating >= 4)].business_idx)

        r = recall_at_k(recs, truth, K)
        n = ndcg_at_k(recs, truth, K)

        if r is not None:
            recalls.append(r)
        if n is not None:
            ndcgs.append(n)

    return np.mean(recalls), np.mean(ndcgs)


In [17]:
recall10, ndcg10 = evaluate_jaccard(K=10)
print("Jaccard Recall@10 =", recall10)
print("Jaccard NDCG@10 =", ndcg10)

Jaccard Recall@10 = 0.0347732181425486
Jaccard NDCG@10 = 0.023778397706640864


## Baseline 3: Popularity-Based Baseline（Most Popular Restaurants）

### Part 0 — Load Data

In [20]:
# Load preprocessed data
data = pickle.load(open("preprocessed_data.pkl", "rb"))

train_df = data["train_df"]
test_df  = data["test_df"]

# Keep necessary fields only
train_df = train_df[["user_idx", "business_idx", "rating"]]
test_df  = test_df[["user_idx", "business_idx", "rating"]]

# Load business stats (optional but recommended)
business_stats = data["business_stats"]  # contains n_reviews & avg_rating


### Part 1 — Compute Popularity

In [21]:
popularity_df = (
    train_df.groupby("business_idx")
            .size()
            .reset_index(name="popularity")
)


### Part 2 —  popularity list

In [22]:
# Sort restaurants by popularity (descending)
popular_items = (
    popularity_df.sort_values("popularity", ascending=False)
                 .business_idx
                 .tolist()
)

### Part 3 — Popularity-Based recommendation function

In [23]:
def recommend_popularity_to_user(user_idx, topK=10):
    seen = set(train_df[train_df.user_idx == user_idx].business_idx)

    recs = []
    for item in popular_items:
        if item not in seen:
            recs.append(item)
        if len(recs) >= topK:
            break
    return recs


### Part 4 — Recall@K / NDCG@K

In [24]:
def recall_at_k(recommended, ground_truth, K):
    if len(ground_truth) == 0:
        return None
    hit = len(set(recommended[:K]) & set(ground_truth))
    return hit / len(ground_truth)


In [25]:
def ndcg_at_k(recommended, ground_truth, K):
    dcg = 0.0
    for rank, item in enumerate(recommended[:K], start=1):
        if item in ground_truth:
            dcg += 1 / math.log2(rank + 1)

    max_rel = min(K, len(ground_truth))
    idcg = sum(1 / math.log2(rank + 1) for rank in range(1, max_rel + 1))

    return dcg / idcg if idcg > 0 else None


### Part 5 — global evaluate_popularity()

In [28]:
def evaluate_popularity(K=10, sample_users=2000):
    recalls = []
    ndcgs = []

    users = test_df.user_idx.unique()
    users = np.random.choice(users, size=min(len(users), sample_users), replace=False)

    for u in users:
        recs = recommend_popularity_to_user(u, topK=K)
        truth = list(test_df[(test_df.user_idx == u) & (test_df.rating >= 4)].business_idx)

        r = recall_at_k(recs, truth, K)
        n = ndcg_at_k(recs, truth, K)

        if r is not None:
            recalls.append(r)
        if n is not None:
            ndcgs.append(n)

    return np.mean(recalls), np.mean(ndcgs)


In [29]:
recall10, ndcg10 = evaluate_popularity(K=10)
print("Popularity Recall@10 =", recall10)
print("Popularity NDCG@10 =", ndcg10)

Popularity Recall@10 = 0.010103768432550519
Popularity NDCG@10 = 0.005506666006273678


## Summary

In [43]:
summary = f"""
============================================================
               BASELINE MODEL COMPARISON SUMMARY
============================================================

We compare three baseline models on the Google Local Restaurants
recommendation task:

1. Popularity-Based Recommendation
2. Item–Item Jaccard Similarity
3. Matrix Factorization (MF)

Evaluation metrics:
- Recall@10
- NDCG@10

===================== Quantitative Results ====================

Popularity:
    Recall@10 = 0.0101
    NDCG@10   = 0.0055

Jaccard Item–Item Similarity:
    Recall@10 = 0.0347
    NDCG@10   = 0.0238

Matrix Factorization (MF):
    Recall@10 = 0.0097
    NDCG@10   = 0.0065

=========================== Analysis ============================

• Jaccard Similarity is the strongest baseline.
  - Recall@10 is over 3× higher than Popularity.
  - Recall@10 is over 3.5× higher than MF.
  - NDCG@10 is also over 3–4× higher than the other baselines.
  Jaccard effectively captures co-liked restaurant patterns even under sparsity.

• MF underperforms due to:
  - Extremely sparse dataset (avg ~3.46 ratings/user)
  - Rating distribution heavily skewed toward 4–5
  - Only 5 training epochs → underfitting
  - MF optimizes MSE, not ranking loss

• Popularity is a weak but meaningful non-personalized baseline.
  Recommended items are identical across all users, limiting performance.

========================== Conclusion ===========================

Item–Item Jaccard Similarity is the best-performing baseline for
Top-K restaurant recommendation on this dataset. It provides a
strong personalized baseline for evaluating more advanced models
such as improved MF, neural collaborative filtering, or
embedding-based recommendation methods.

============================================================
"""

print(summary)



               BASELINE MODEL COMPARISON SUMMARY

We compare three baseline models on the Google Local Restaurants
recommendation task:

1. Popularity-Based Recommendation
2. Item–Item Jaccard Similarity
3. Matrix Factorization (MF)

Evaluation metrics:
- Recall@10
- NDCG@10

===================== Quantitative Results ====================

Popularity:
    Recall@10 = 0.0101
    NDCG@10   = 0.0055

Jaccard Item–Item Similarity:
    Recall@10 = 0.0347
    NDCG@10   = 0.0238

Matrix Factorization (MF):
    Recall@10 = 0.0097
    NDCG@10   = 0.0065

=========================== Analysis ============================

• Jaccard Similarity is the strongest baseline.
  - Recall@10 is over 3× higher than Popularity.
  - Recall@10 is over 3.5× higher than MF.
  - NDCG@10 is also over 3–4× higher than the other baselines.
  Jaccard effectively captures co-liked restaurant patterns even under sparsity.

• MF underperforms due to:
  - Extremely sparse dataset (avg ~3.46 ratings/user)
  - Rating dis

### Analysis
### Jaccard Similarity is the strongest baseline.

It achieves the best performance on both metrics, outperforming popularity by more than 3× and MF by more than 3.5×.
This demonstrates that item–item collaborative filtering based on co-liked restaurants captures useful similarity signals, even in a sparse dataset.

### MF performs worse than expected.

Although MF is a powerful model for explicit-feedback recommendation, its performance is limited in this dataset due to:

Severe sparsity (average only 3.46 ratings per user)

Highly skewed rating distribution (most ratings are 4–5)

Insufficient training epochs

MF optimizing MSE instead of ranking loss

As a result, MF embeddings are under-trained and less effective for top-K ranking compared to Jaccard.

Popularity is a weak but meaningful non-personalized baseline.

Because recommendations are identical for all users, this baseline struggles to match the personalized approaches.
However, it provides an important lower bound for model performance.

### Conclusion

Among the three baselines, Item–Item Jaccard Similarity is clearly the best performing model for top-K restaurant recommendation on this dataset.
It serves as a strong personalized baseline and a meaningful reference point for more advanced models such as improved MF, neural collaborative filtering, or embedding-based methods.